# This repository holds both the code to create catalogues & its documentation

## Data Init
Function that initializes the data, creating a new h5 file or accessing one.

## Detect
Function that graphs a section of the data

## CatalogueClassic
Function that creates a catalogue given parameters

## PlotEvent
Function that graphs an event from a catalogue

##


In [67]:
"""
Imported modules, has lots of useful stuff. I should give a short description of the more esoteric ones maybe.

"""
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import datetime
import pickle #dictionaries!
import scipy
from scipy import signal #For data cleanup
import glob #Create a list of files
import time
from collections import defaultdict
import obspy
from obspy.core import UTCDateTime #MAYBE useful.
from obspy.core import read
from obspy.signal import trigger #For triggering, required.
import os
import emcee
import tqdm
import matplotlib.dates as mdates


# Functions

## Data Interaction Functions (Users use these)

In [197]:
def DataInit(DASLoc, DASLink, Positionstxt):
    
    """  
    Initializes the data & sets global variables. If done on a file that already exists, deletes the file and makes it again.
    Creates an h5 file with group "Data" groups 'PosData','TimeData', and 'RootInfo'.

    
    Parameters:
    - DASLoc: Name of the h5 file to be made with this data. String.
    - DASLink: A directory string to the Das data, USE GLOB.
    - Positionstxt: A txt file containing the physical coordinates of the channel locations. Latitude & Longitude related.
    
    Returns nothing
    """
    if os.path.exists("my_file.h5"): #Checks to see if file exists, if it does only sets it to the file.
        print("File exists, setting.")
        global currentfile_NY #Sets current file global
        currenfile_NY = DASLoc
        with h5.File(DASLoc, 'r') as PulledFIle:
            X = PulledFile['Data']['Cable']['CableX'][:]
            Y = PulledFile['Data']['Cable']['CableY'][:]
            Z = PulledFile['Data']['Cable']['CableZ'][:]
            global channel_locations_NY  #Sets channel locations global
            channel_locations = np.array([X,Y,Z])
        
    else: #If the file doesn't exist, it creates it.
        print("File does not exist, creating.")
        Data_1 = glob.glob(DASLink) #Acquires a glob of the data
        Data_1 = sorted(Data_1)
        for file in Data_1: #This for loop removes corrupted files.
            try:
                with h5.File(file, 'r') as f:
                    f.keys()
            except (OSError, IOError) as e:
                print(f"Corrupted file: {file} - Error: {e}")
                Data_1.remove(file)
    
        with h5.File(DASLoc, 'w') as Newfile: #Writes a new file, this will overwrite prior files. I should make a delete funciton...
            GData = Newfile.create_group('Data/PosData') #Creates data files, linking to DASLink
            TData = Newfile.create_group('Data/TimeData')
            Newfile['Data']['RootInfo'] = h5.ExternalLink(Data_1[0],'/')
            
            
            #Below for loops turn them into numbered sections, should these be datetime related instead?
            
            for i in range(len(Data_1)): #Turns them into numbered sections.
                GData[f'Section {i}'] = h5.ExternalLink(Data_1[i],'/Acquisition/Raw[0]/RawData/')
                i+=1
            for i in range(len(Data_1)):
                TData[f'Section {i}'] = h5.ExternalLink(Data_1[i],'/Acquisition/Raw[0]/RawDataTime/')
                i+=1
        AddingPositionData(DASLoc,Positionstxt) #Adds position data for cable from a txt file, sets global.
            
        global currentfile_NY #Sets current file global
        currentfile_NY = DASLoc
    global currentcatalogue_NY #Just does this for ease of not getting weird errors.
        
    
            
            
        #I also want this to set a lot of other globals that are important: Like cable position. EX:
        
        
            
            

In [586]:
def Catalogue_Classic(CatalogueName,Sections,Low,High,STAn,LTAn,Dataset=currentfile_NY,TrigOn=2.5,TrigOff=1.5,tdownmax=20, tupevent=60, thr1=5, thr2=10, preset_len=250, p_dur=250,loopj=0,loopk=0,loopm=0):
    """
    Catalogues Triggers into a Datasets "Catalogue" group.
    
    Parameters:
    - CatalogueName: The name of the Catalogue that is being created.
    - Sections: Desired sections to plot.
    - Low: Lowpass filter
    - High: Highpass filter
    - STAn: The bound of detection.
    - LTAn: The bound of reset.   
    - Dataset: Link to the Dataset
    
    Please note that the Total, Positives, and Negatives attribute are non-inclusive, (total 255 means there are events 0-254)
    
    Returns nothing
    
    """
    HARDCODED_ERROR_AVOIDER = 100 #Unhardcode this sometime. 150 works well, but is not perfect.
    
    
    for i in np.arange(Sections[0],Sections[1]+1,1):

        if os.path.exists(currentfile_NY):
            None
        else:
            print("ERROR! Dataset not found. Please Initialize")
            return None

        with h5.File(f'{Dataset}','r') as PulledFile:   #Acquires relevant data from the file.
            DATA_TEMP = PulledFile['Data']['PosData'][f'Section {i}']
            DATA_TEMP = DATA_TEMP[:,:]
            Pulserate = PulledFile['Data']['RootInfo']['Acquisition'].attrs['PulseRate']

                        #These create a list of possible events in a section.
            
        Triggerlist = Triggering(Dataset,i,Low,High,STAn,LTAn,TrigOn,TrigOff) #Generates list of triggers per section
        CheckedTriggerlist = find_overlapping_triggers(Triggerlist) #Checks them for weirdness
        EventData = group_triggers_into_events(CheckedTriggerlist,Pulserate,min_triggers=60) #Creates coherent group matrices (list of [3,:] arrays, not homogenous.)


                        #These loops check if the triggered values might be error.
        Keeplist=[]
        Overheatlist=[]
        MADlist=[]
        if EventData != []:
            for p in EventData:
                Keep,MAD = Variance_Check(p, HARDCODED_ERROR_AVOIDER)
                Keeplist.append(Keep)
                MADlist.append(MAD)
                #if Keep != False:
                NotOverheat = Overheat_Check(p,Dataset,CatalogueName,i)
                Overheatlist.append(NotOverheat)
                #else: NotOverheat = True
        else:
            Keep=True
            MAD=0
            NotOverheat=True

        PulledFile.close()
        with h5.File(f'{Dataset}', 'a') as PulledFile:            #This sections should, theoretically, set the Events in the catalogue seperated by event.    
            if loopm == 0:
                if f'Catalogues/{CatalogueName}' in PulledFile:

                    #del PulledFile[f'Catalogues/{CatalogueName}/Eventlist']
                    #del PulledFile[f'Catalogues/{CatalogueName}/Errorlist'] #This is unnecessary I think
                    del PulledFile[f'Catalogues/{CatalogueName}']
                    Cath5 = PulledFile.create_group(f'Catalogues/{CatalogueName}')
                    #Errorh5 = PulledFile.create_group(f'Catalogues/{CatalogueName}/Errorlist')
                    PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Positives'] = 0
                    PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Total'] = 0
                    PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Negatives'] = 0

                else:
                    Cath5 = PulledFile.create_group(f'Catalogues/{CatalogueName}')
                    #Errorh5 = PulledFile.create_group(f'Catalogues/{CatalogueName}/Errorlist')
                Quickh5 = PulledFile['Catalogues'][CatalogueName]
                Quickh5.attrs['Lowpass'] = Low
                Quickh5.attrs['Highpass'] = High
                Quickh5.attrs['STAn'] = STAn
                Quickh5.attrs['LTAn'] = LTAn
                Quickh5.attrs['TrigOn'] = TrigOn
                Quickh5.attrs['TrigOff'] = TrigOff 
                Quickh5.attrs['tdownmax']=tdownmax
                Quickh5.attrs['tupevent']=tupevent 
                Quickh5.attrs['thr1']=thr1
                Quickh5.attrs['thr2']=thr2
                Quickh5.attrs['preset_len']=preset_len
                Quickh5.attrs['p_dur']=p_dur
                Quickh5.attrs['MAD'] = MAD
                loopm+=1

                #Post = PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Positives'] 
                #Totl = PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Total'] 
                #Negt = PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Negatives'] 

            else:
                Cath5 = PulledFile['Catalogues'][CatalogueName]
                #Errorh5 = PulledFile['Catalogues'][CatalogueName]['Errorlist']        

            #if Section%100 == 0:
            print(f"Just started Section {i}")
            



            if EventData!=[]:
                for g in range(len(EventData)):  #So this makes a dataset, it's different than the below function. Maybe make them more the same, dataset within another group? :)
                    
                    MakeEventGroup = Cath5.create_group(f"Event {loopm-1}") #The -1 exists so that it starts at event 0, this can be changed.
                    SetEvent = MakeEventGroup.create_dataset("Triggers", data=EventData[g])
                    if Keeplist[g] == False or Overheatlist[g] == False:
                        MakeEventGroup.attrs['Error'] = True
                        loopk+=1
                    else:
                        loopj+=1
                        MakeEventGroup.attrs['Error'] = False
                    MakeEventGroup.attrs['Section'] = i 
                    eventstart, eventend = RangeCalc(EventData[g],Pulserate)
                    MakeEventGroup.attrs['Range'] = (eventstart,eventend)
                    loopm+=1
                    
                    """   BROADLY REMOVED
                if Keeplist[g] == False or Overheatlist[g] == False:
                    ErrorSet = Errorh5.create_dataset(f'Error {loopk}', data = EventData[g]) #Give this a similar functionality to the below loop.
                    loopk+=1
                    loopm+=1
                    errorstart, errorend = RangeCalc(EventData[g],Pulserate)
                    ErrorSet.attrs['Range'] = (errorstart,errorend)
                    ErrorSet.attrs['Section'] = i
                else:
                    MakeEventGroup = Cath5.create_group(f"Event {loopj}") #Adds it to the catalogue

                    loopj+=1 # #'s events
                    loopm+=1
                    SetEvent = MakeEventGroup.create_dataset("Triggers", data=EventData[g])
                    MakeEventGroup.attrs['Section'] = i #Sets the section of the event.
                    #Pwaves = Baer_Picker(EventData[g],DATA_TEMP,Pulserate,tdownmax, tupevent, thr1, thr2, preset_len, p_dur)
                    #Pwaves = AICPicker(h,DATA_TEMP,Pulserate)
                    #print(Pwaves)
                    #print(type(Pwaves))
                    #print(Pwaves.shape)
                    #SetPicks = MakeEventGroup.create_dataset("P_waves", data=Pwaves)


                    #if h[0,1]-(3*Pulserate) < 0:  #This all keeps it from going out of bounds.
                    #    eventstart = 0
                    #else:
                    #    eventstart = h[0,1]-(1*Pulserate)
                    #if h[-1,2]+(3*Pulserate) > (60*Pulserate):
                    #    eventend = 60*Pulserate
                    #else:
                    #    eventend = h[-1,2]+(2*Pulserate)
                    eventstart, eventend = RangeCalc(EventData[g],Pulserate)
                    MakeEventGroup.attrs['Range'] = (eventstart,eventend) #Sets the range that can be called in a graphing function.
                    #SetEvent.attrs['move out, picks, whatever'] Do this later nathan, probably important! For now this works.
"""
            else:
                None
            print(f"{loopm-1} events so far, {loopj} positive and {loopk} negative")
            
    with h5.File(f'{currentfile_NY}', 'a') as PulledFile: #Updates counts, if the data init is adding to a catalogue
            PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Positives'] = PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Positives'] + loopj
            PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Total'] = PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Total'] + loopm-1
            PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Negatives'] = PulledFile['Catalogues'][f'{CatalogueName}'].attrs['Negatives'] + loopk
            
    
    global currentcatalogue_NY
    currentcatalogue_NY = CatalogueName
            
    #OK SO:
        #We generate the list of "points" as individual events in the catalogue
            #We do this by having a for loop for i in events, where is just saves that particular section as a dataset titled "Event #" in the catalogue directed.
        #We save attributes to each event, including:
            #Section
            #Time in this section
            #Maximum Amplitude? 
            #Move out?
            # # or types of waves
        #If I can manage it having picks for p and s waves would be nice too.



In [572]:
def Detect(Section,Dataset=currentfile_NY,Sensitivity=.05,Lowpass=1,Highpass=50,YLowerLim=None,YUpperLim=None, doseconds = True, show=True):
    """
    Generates a pcolormesh of the desired Section with a designated parameters
        
    Parameters:
    - Section: The section to plot
    - Dataset: The file location
    - Sensitivity: The sensitivity (in radians)
    - Lowpass: for Lowpass filter (Hz)
    - Highpass: for Highpass filter (Hz)
    - YLowerLim: Lower limit (s)
    - YUpperLim: Upper limit (s)
    - doseconds: If true, then the time axis is in seconds, if false it's datetime. 
    - show: Whether the plotted image should be shown.
    
    Returns nothing
        
    """
    #TO DO: Add an input to allow changing of ACTUAL FILTERING, not just highpass/lowpass.
    #TO DO: Make limits a tuple for simplification.
    #TO DO: Allow option to convert or not to convert to seconds, specifically for p wave picks... I need datetimes.
    
    Smin = -Sensitivity
    Smax = Sensitivity
    if YLowerLim == None:
        YLowerLim = 0
    if YUpperLim == None:
        YUpperLim = 60
    with h5.File(f'{Dataset}','r') as PulledFile: #WARNING: Generalize this, so that it can be from the "selected" catalogue
        DATA_TEMP = PulledFile['Data']['PosData'][f'Section {Section}']
        TIME_TEMP = PulledFile['Data']['TimeData'][f'Section {Section}']
        INFO = PulledFile['Data']['RootInfo']
        PR = INFO['Acquisition'].attrs['PulseRate']
    
        channels = np.arange(0,DATA_TEMP.shape[1],1)
        seconds = np.arange(0,DATA_TEMP.shape[0],1)
        Time = sintela_to_datetime(TIME_TEMP[:,]) 
        
    Seconds = [(T-Time[0]).total_seconds() for T in Time] #Super curious list comprehension function, I don't really get how it works but it's neat, "Vectorizes" the set of data and subtracts the time from each, which produces an array of seconds.
    #Seconds = [x for x in Seconds if YLowerLim <= x <= YUpperLim]

    sos = scipy.signal.butter(10, [Lowpass,Highpass], 'bp', fs = PR, output='sos') #WARNING WARNING: INFO['Acquisition'] IS NOT A GENERALIZED FUNCTION, LACK OF VERSATALITY!!!!
    filtered = signal.sosfiltfilt(sos,DATA_TEMP,axis=0)

    Seconds = np.round(np.array(Seconds),4)
    CorrectedLowerLim = np.where(Seconds == float(YLowerLim))
    CorrectedUpperLim = np.where(Seconds == float(YUpperLim))
    n = YLowerLim
    m = YUpperLim
    count = 0
    max_iters = 20
    while CorrectedLowerLim[0].size == 0:
        n = n - .01
        CorrectedLowerLim = np.where(Seconds == float(n))
        count += 1
        if count >= max_iters:
            n = YLowerLim
            count = 0
            while CorrectedLowerLim[0].size == 0:
                n = n + .01
                CorrectedLowerLim = np.where(Seconds == float(n))
                count +=1
                if count >= max_iters:
                    break
        if count >= max_iters:
            CorrectedLowerLim = (np.array([0]),)
            break
            
    count = 0
    while CorrectedUpperLim[0].size == 0:
        m = m + .01
        CorrectedUpperLim = np.where(Seconds == float(m))
        count += 1
        if count >= max_iters:
            m = YUpperLim
            count = 0
            while CorrectedUpperLim[0].size == 0:
                m = m - .01
                CorrectedUpperLim = np.where(Seconds == float(m))
                count +=1
                if count >= max_iters:
                    break
        if count >= max_iters:
            CorrectedUpperLim = (np.array([120000]),)
            break
    
    if doseconds == True:
        
    
        Seconds = Seconds[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0]]
        
        filtered = filtered[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0],:]
        
        fig,ax = plt.subplots()
        smesh = ax.pcolormesh(channels, Seconds, filtered, vmin = Smin, vmax = Smax, cmap = 'seismic',shading='nearest')
        plt.xlabel('channels')
        plt.ylabel('seconds')
        #ax.set_ylim(YLowerLim, YUpperLim)
        fig.colorbar(smesh,ax=ax)

    else:
        Time = Time[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0]]
        filtered = filtered[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0],:]
        print(f"Time shape is {Time.shape}")
        print(f"Filltered shape is {filtered.shape}")
        fig,ax = plt.subplots()
        smesh = ax.pcolormesh(channels,Time,filtered, vmin = Smin, vmax = Smax, cmap = 'seismic',shading='nearest')
        plt.xlabel('channels')
        plt.ylabel('datetime')
        #ax.yaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S.%f')) 
        #ax.set_ylim(YLowerLim, YUpperLim)
        bar = fig.colorbar(smesh,ax=ax)
        bar.set_label("phase shift (radians)")
    if show==True:
        plt.show()
    return


In [551]:
def setcatalogue(Catalogue):
    """
    Sets the global 'currentcatalogue_NY' as a string, for easier function calling.
    
    Parameters:
    - Catalogue: catalogue you want to set as currentcatalogue_NY
    
    Returns nothing
    """
    global currentcatalogue_NY
    currentcatalogue_NY = Catalogue
    #Sets a catalogue as a global.
    
def setfile(File):
    """
    Sets the global 'currentfile_NY' as a string, for easier function calling.
    
    Parameters:
    - File: file path you want to set as currentfile_NY
    
    Returns nothing
    """
    global currentfile_NY
    currentfile_NY = File
    
def setcoords(Positionstxt):
    """
    Sets the global 'channel_locations' as a nested array of [x,y,z] for each channel.
    
    Parameters:
    - Positionstxt: a txt file of 3 columns (x,y,z) to be set as the hlobal channel_locations_NY 
    
    Returns nothing
    """ #Hey Nathan why is this global having a _ between two words, but not the others? Ahh I'll fix it later.
    global channel_locations_NY
    
    data = np.loadtxt(f"{Positionstxt}")
    X = data[:, 0]
    Y = data[:, 1]
    Z = data[:, 2]
        
    channel_locations_NY = [X,Y,Z]

In [627]:
def PlotEvent(Event,Catalogue=currentcatalogue_NY,Dataset=currentfile_NY,Sensitivity=.05,Lowpass = None,Highpass = None,ShowTrig=False,ShowPick=False,Type=None, savefigs = False, folder=None,doseconds=True): #Plots an event from a catalogue.
    """
    Plots an event from the catalogue using its range.
    
    Parameters:
    - Event: Event to plot
    - Catalogue: Catalogue to take event from
    - Dataset: Location of file
    - Sensitivity: The sensitivity (in radians)
    - Lowpass: for Lowpass filter (Hz)
    - Highpass: for Highpass filter (Hz)
    - ShowTrig: Show the triggers?
    - ShowPick: Show the automatic picks? (CURRENTLY DEPRECATED)
    - Type: Positives, Negatives, All?
    - savefigs: should figures be saved in the given path to folder?
    - folder: path to folder to save figures to
    - doseconds: If true, then the time axis is in seconds, if false it's datetime. 
    
    Returns nothing
    
    """
    with h5.File(Dataset,'r') as PulledFile:
        Pulserate = PulledFile['Data']['RootInfo']['Acquisition'].attrs['PulseRate']
        
        if Lowpass == None:
            Lowpass = PulledFile['Catalogues'][Catalogue].attrs['Lowpass']
        if Highpass == None:
            Highpass = PulledFile['Catalogues'][Catalogue].attrs['Highpass']        
    
    
        MyEvent = PulledFile['Catalogues'][Catalogue][f'Event {Event}']
        Section = PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Section']
        Range = PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Range']
        time = PulledFile['Data']['TimeData'][f'Section {Section}']
        Timeinit = sintela_to_datetime(time[0])
        doplot = None
       
        if Type == None:
            doplot = True
        elif Type == 'positive':
            if PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Error'] == False:
                doplot = True
            else:
                doplot = False
        elif Type == 'negative':
            if PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Error'] == True:
                doplot = True
            else:
                doplot = False
        else:
            print("Nonvalid Type Parameter")
        if doplot == True:
            Detect(Section,Sensitivity = Sensitivity,Lowpass = Lowpass,Highpass = Highpass,YLowerLim=(Range[0]/Pulserate),YUpperLim=(Range[1]/Pulserate),show=False,doseconds=doseconds)
            if ShowTrig==True:
                MyTrig = MyEvent['Triggers']
                MyTrig = MyTrig[:,:]
                plt.scatter(MyTrig[:,0],MyTrig[:,2]/Pulserate,s=7, alpha=.5)
                plt.scatter(MyTrig[:,0],MyTrig[:,1]/Pulserate,s=7, alpha=.5)
            if ShowPick==True:
                MyPick = MyEvent['P_waves']
                MyPick = MyPick[:,:]
                plt.scatter(MyPick[:,0],MyPick[:,1]/Pulserate,s=7, alpha=1)
            fig = plt.gcf()
            plt.title(f"Event {Event} in section {Section}, at {Timeinit}")
            plt.show()
            if savefigs == True:
                os.makedirs(folder, exist_ok=True) 
                save_path = os.path.join(folder, f"Event_{Event}.png")
                plt.savefig(save_path)
                print("Saved!")
        

        
        #It needs to print
            #A. The section given by 'range'
            #B. The trigger points as leetle dots that don't get in the way of the graph.

In [654]:
def change_event(event, ertype = None,Dataset=currentfile_NY,Catalogue=currentcatalogue_NY):
    """
    Change the events error attribute from positive to negative or vice versa
    
    Parameters:
    - event: the event to change
    - ertype: True is if it is error, False is if it is not.
    - Datset: current file
    - Catalogue: current catalogue 
    
    Returns nothing
    """
    with h5.File(f'{Dataset}','a') as PulledFile:
        temppos = PulledFile['Catalogues'][f'{Catalogue}'].attrs['Positives']
        tempneg = PulledFile['Catalogues'][f'{Catalogue}'].attrs['Negatives']
        if ertype==None:
            return
        elif ertype==True:
            if PulledFile['Catalogues'][f'{Catalogue}'][f'Event {event}'].attrs['Error'] == True:
                return
            PulledFile['Catalogues'][f'{Catalogue}'][f'Event {event}'].attrs['Error'] = True
            temppos = temppos-1
            tempneg = tempneg+1
        elif ertype==False:
            if PulledFile['Catalogues'][f'{Catalogue}'][f'Event {event}'].attrs['Error'] == False:
                return
            PulledFile['Catalogues'][f'{Catalogue}'][f'Event {event}'].attrs['Error'] = False
            temppos = temppos+1
            tempneg = tempneg-1
        else:
            print("This is a boolean, true or false the error.")
        PulledFile['Catalogues'][f'{Catalogue}'].attrs['Positives'] = temppos
        PulledFile['Catalogues'][f'{Catalogue}'].attrs['Negatives'] = tempneg

            
def autochange(Dataset=currentfile_NY,Catalogue=currentcatalogue_NY):
    """
    Only do this after taking Pwave picks, marks any "skipped" events as error (no wave picks taken)
    
    Parameters:
    - Dataset: current file
    - Catalogue: current catalogue
    
    Returns nothing
    """
    with h5.File(f'{Dataset}','a') as PulledFile:
        for i in PulledFile['Catalogues'][f'{Catalogue}']:
            pwav = PulledFile['Catalogues'][f'{Catalogue}'][i]['P_waves'][:]
            if pwav.size == 0:
                event = i.split()[1] 
                change_event(event, ertype = True,Dataset=Dataset,Catalogue=Catalogue)
                #i.attrs['Error']=True
            else:
                None
                #i.attrs['Error']=False
                #change_event(event, ertype = False,Dataset=Dataset,Catalogue=Catalogue)
            
                
                
def totalevents(Dataset=currentfile_NY,Catalogue=currentcatalogue_NY):
    "prints the total # of events"
    with h5.File(f'{Dataset}','r') as PulledFile:
        print(PulledFile['Catalogues'][f'{Catalogue}'].attrs['Total'])

        
def totalpositives(Dataset=currentfile_NY,Catalogue=currentcatalogue_NY):
    "prints the number of positive events"
    with h5.File(f'{Dataset}','r') as PulledFile:
        print(PulledFile['Catalogues'][f'{Catalogue}'].attrs['Positives'])
    
    
def totalnegatives(Dataset=currentfile_NY,Catalogue=currentcatalogue_NY):
    "prints the number of negative events"
    with h5.File(f'{Dataset}','r') as PulledFile:
        print(PulledFile['Catalogues'][f'{Catalogue}'].attrs['Negatives'])
    

In [471]:
def get_subsection(Event,Dataset=currentfile_NY,Catalogue=currentcatalogue_NY,Sensitivity=.05,Lowpass=1,Highpass=50):
    """
    Returns
    
    
    """
    
    Smin = -Sensitivity
    Smax = Sensitivity
    with h5.File(f'{Dataset}','r') as PulledFile: #WARNING: Generalize this, so that it can be from the "selected" catalogue
        DATA_TEMP = PulledFile['Data']['PosData'][f'Section {Event}']
        TIME_TEMP = PulledFile['Data']['TimeData'][f'Section {Event}']
        INFO = PulledFile['Data']['RootInfo']
        PR = INFO['Acquisition'].attrs['PulseRate']
    
        channels = np.arange(0,DATA_TEMP.shape[1],1)
        seconds = np.arange(0,DATA_TEMP.shape[0],1)
        Time = sintela_to_datetime(TIME_TEMP[:,]) 
        
        MyEvent = PulledFile['Catalogues'][Catalogue][f'Event {Event}']
        Section = MyEvent.attrs['Section']
        Range = MyEvent.attrs['Range']
        pwaves = MyEvent['P_waves'][:].T
        
        YLowerLim = (Range[0])/PR
        YUpperLim = (Range[1])/PR
        
        #print(DATA_TEMP)
        
    Seconds = [(T-Time[0]).total_seconds() for T in Time] #Super curious list comprehension function, I don't really get how it works but it's neat, "Vectorizes" the set of data and subtracts the time from each, which produces an array of seconds.
    #Seconds = [x for x in Seconds if YLowerLim <= x <= YUpperLim]
    
    sos = scipy.signal.butter(10, [Lowpass,Highpass], 'bp', fs = PR, output='sos') #WARNING WARNING: INFO['Acquisition'] IS NOT A GENERALIZED FUNCTION, LACK OF VERSATALITY!!!!
    filtered = signal.sosfiltfilt(sos,DATA_TEMP,axis=0)

    Seconds = np.round(np.array(Seconds),4)
    
    
    #This entire section is just my jank "Recovery" code that does its very best to fix data splicing when there's weird limits.
    
    CorrectedLowerLim = np.where(Seconds == float(YLowerLim))
    CorrectedUpperLim = np.where(Seconds == float(YUpperLim))
    n = YLowerLim
    m = YUpperLim
    count = 0
    max_iters = 20
    while CorrectedLowerLim[0].size == 0:
        n = n - .01
        CorrectedLowerLim = np.where(Seconds == float(n))
        count += 1
        if count >= max_iters:
            n = YLowerLim
            count = 0
            while CorrectedLowerLim[0].size == 0:
                n = n + .01
                CorrectedLowerLim = np.where(Seconds == float(n))
                count +=1
                if count >= max_iters:
                    break
        if count >= max_iters:
            CorrectedLowerLim = (np.array([0]),)
            break
            
    count = 0
    while CorrectedUpperLim[0].size == 0:
        m = m + .01
        CorrectedUpperLim = np.where(Seconds == float(m))
        count += 1
        if count >= max_iters:
            m = YUpperLim
            count = 0
            while CorrectedUpperLim[0].size == 0:
                m = m - .01
                CorrectedUpperLim = np.where(Seconds == float(m))
                count +=1
                if count >= max_iters:
                    break
        if count >= max_iters:
            CorrectedUpperLim = (np.array([120000]),)
            break
    
    
    Seconds = Seconds[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0]]

    filtered = filtered[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0],:]
        
        
    #Need to make sure this can be done in datetime, not seconds.
    return pwaves,filtered
    
    

In [664]:
def categorize(event,string,integer = None,catalogue=currentcatalogue_NY,data=currentfile_NY):
    """
    Parameters:
    - event: the event to categorize.
    - string: the name of the new attribute.
    - integer: If there's a number associated.
    - catalogue: relevant catalogue.
    - data: relevant file.
    
    Returns nothing
    """
    with h5.File(f'{data}','a') as PulledFile:
        Attradd = PulledFile['Catalogues'][f'{catalogue}']['EventList'][f'Event {event}'].attrs
        Attradd[f'{string}'] = integer

        
def describeevents(Events,Dataset=currentfile_NY,Catalogue=currentcatalogue_NY):
    """
    Placeholder function to describe events.
    
    Parameters:
    - Events: Tuple of events that dictates the range over which to describe.
    - Dataset: current file
    - Catalogue: current catalogue
    
    Returns nothing
    """
    Start = Events[0]
    End = Events[1]
    with h5.File(Dataset,"r") as PulledFile:
        CurrentCat = PulledFile['Catalogues'][Catalogue]
        for i in range(Start,End):
            quicklist = []
            quallist = []
            for j in CurrentCat[f'Event {i}'].attrs:
                quicklist.append(j)
                quallist.append(CurrentCat[f'Event {i}'].attrs[j])
            print(f"Event {i} has the attributes {quicklist} with the values {quallist}")

## Data Processing Functions

In [593]:
def sintela_to_datetime(sintela_times):
    '''
    Turns DAS time data into a datetime object.
    
    Parameters:
    - sintela_times: Numpy Array of time data
    
    Returns:
    - an array of datetime.datetime 
    
    Function provided by John-Morgan
    
    ''' 
    days1970 = datetime.date(1970, 1, 1).toordinal() #total days from 0

    # Vectorize everything
    converttime = np.vectorize(datetime.datetime.fromordinal)
    addday_lambda = lambda x : datetime.timedelta(days=x)
    adddays = np.vectorize(addday_lambda )
    
    day = days1970 + sintela_times/1e6/60/60/24 #Convert Sintela times to days from microseconds
    thisDateTime = converttime(np.floor(day).astype(int))
    dayFraction = day-np.floor(day)
    thisDateTime = thisDateTime + adddays(dayFraction)

    return thisDateTime


def Triggering(Dataset,Section,Low,High,STAn,LTAn,TrigON=3,TrigOFF=2):
    """
    Find the triggers in a section.
    
    Parameters:
    - Dataset: Link to the Dataset
    - Section: The desired Section(s? maybe eventually?)
    - Low: Lowpass filter
    - High: Highpass filter
    - STAn: The bound of detection.
    - LTAn: The bound of reset.
    
    Returns:
    - ListoTriggers: a list of arrays in the shape of (N, 2) with [start_time, end_time] at each N. 
 
    """ 
        
    with h5.File(f'{Dataset}','r') as PulledFile:
        DATA_TEMP = PulledFile['Data']['PosData'][f'Section {Section}']
        INFO = PulledFile['Data']['RootInfo']
        Pulserate = INFO['Acquisition'].attrs['PulseRate']
        npts = len(DATA_TEMP[:,0])
        starttime = DATA_TEMP.attrs['PartStartTime']
        STAsamples=int(STAn*Pulserate)
        LTAsamples=int(LTAn*Pulserate)
        
        Sos = scipy.signal.butter(10, [Low,High], 'bp', fs = Pulserate, output='sos') #Filtering the data
        Filtered = signal.sosfiltfilt(Sos,DATA_TEMP,axis=0)
        
        cft = np.apply_along_axis(obspy.signal.trigger.classic_sta_lta, 0,Filtered,STAsamples,LTAsamples) #Vectorizes and applys trigger.
        
        ListoTriggers = []
        for i in range(Filtered[0,:].size):
            
            FilteredChannel = Filtered[:,i]
            MyTrig = trigger.trigger_onset(cft[:,i],TrigON,TrigOFF) #adds the onset of each trigger to the list. 
            
            
            
            if len(MyTrig) > 0:
            # Remove triggers too close to the boundaries
                Valid = (MyTrig[:, 0] > LTAsamples) & (MyTrig[:, 0] < cft[:,0].size - LTAsamples)
                MyTrig = MyTrig[Valid]
                if len(MyTrig) > 0:    
                    ListoTriggers.append(MyTrig)#Divide by Pulserate to get seconds.
                else:
                    ListoTriggers.append(np.empty((0, 2), dtype=np.int64))
            else:
                ListoTriggers.append(np.empty((0, 2), dtype=np.int64))
    
    return ListoTriggers
    
    
def find_overlapping_triggers(all_trigs, start_index=0, min_duration=2):
    """
    Find overlapping triggers between neighboring channels. Function provided largely by Dominik.

    Parameters:
    - all_trigs: list of arrays, each of shape (N, 2) with [start_time, end_time] for one channel
    - start_index: integer offset to apply to channel numbers (default 0)
    - min_duration: minimum number of samples a trigger must span (default 2)

    Returns:
    - overlapping_triggers: array of [channel, start_time, end_time] where neighboring channels overlap
    """

    # Stack all triggers into a single array with channel numbers
    all_trigs_arr = np.vstack([
        np.hstack((
            np.full((trigs.shape[0], 1), i + start_index),  # channel numbers
            trigs
        ))
        for i, trigs in enumerate(all_trigs)
    ])

    # Remove short triggers
    all_trigs_arr = all_trigs_arr[all_trigs_arr[:, 2] - all_trigs_arr[:, 1] >= min_duration]

    # Group triggers by channel
    channel_dict = defaultdict(list)
    for i, row in enumerate(all_trigs_arr):
        chan, start, end = row
        channel_dict[chan].append((start, end, i))  # include index

    # Find overlapping triggers across neighboring channels
    overlap_indices = set()
    channels = sorted(channel_dict.keys())

    for chan in channels:
        if (chan + 1) in channel_dict:
            for start1, end1, idx1 in channel_dict[chan]:
                for start2, end2, idx2 in channel_dict[chan + 1]:
                    if start1 <= end2 and end1 >= start2:
                        overlap_indices.update([idx1, idx2])

    # Return only overlapping triggers
    return all_trigs_arr[sorted(overlap_indices)]



def group_triggers_into_events(triggers, fs, window=0.2, min_triggers=50):
    """
    Groups triggers into events based on time proximity and deduplicates per channel. Function provided largely by Dominik.

    Parameters:
    - triggers: array of shape (N, 3), each row [channel, start_time, end_time]
    - fs: ?
    - window: maximum time difference between consecutive triggers to consider as same event
    - min_triggers: minimum number of triggers required for an event to be kept

    Returns:
    - unique_events: list of ndarrays, each of shape (M, 3), representing one event's unique triggers
    """
    if len(triggers) == 0:
        return []

    # Sort by start time
    triggers = triggers[np.argsort(triggers[:, 1])]

    # Group triggers into events
    events = []
    current_event = [triggers[0]]

    for i in range(1, len(triggers)):
        last_trigger = current_event[-1]
        this_trigger = triggers[i]

        if this_trigger[1] - last_trigger[1] <= window*fs:
            current_event.append(this_trigger)
        else:
            events.append(np.array(current_event))
            current_event = [this_trigger]

    if current_event:
        events.append(np.array(current_event))

    # Filter out small events
    events = [evt for evt in events if len(evt) >= min_triggers]

    # Deduplicate per channel in each event
    unique_events = []
    for evt in events:
        seen_channels = set()
        unique_triggers = []

        for trig in evt:
            channel = trig[0]
            if channel not in seen_channels:
                unique_triggers.append(trig)
                seen_channels.add(channel)

        unique_events.append(np.array(unique_triggers))

    return unique_events

def Variance_Check(TriggerList, VarianceMin): #COMPLETED, LEAVE FOR NOTES.
    """
    A function that checks Mean Absolute Derivation (MAD) of the triggers returned by group_triggers_into_events
    
    Parameters:
    - TriggerList: array of triggers, exists as an array of [M,3], wheren [M,1] is the onset (I believe).
    - VarianceMin: integer, minimum value MAD can be before it's considered omnichannel error.
    
    Returns:
    - Errorcheck: true if not error, false if so.
    - MAD: Value of MAD, mostly used in checking if the parameters are set right. 
    """
    
    #TriggerList will be a list of [0/1/2,:] arrays, where [0,:] is the channel, [1,:] is trig on, [2,:] is trig off.
    #We can use trig on or trig off, unsure which is better.
    #High Variance is GOOD, determine a minimum variance to seperate Seismic events from omnichannel error.
    
    Variance = 0
    
    Median_Trigon = np.median(TriggerList[:,1])

    Abs_Differences = np.abs(TriggerList[:,1]-Median_Trigon)

    MAD = np.median(np.sort(Abs_Differences))
    
    #print(f"Event with Variance {MAD} detected!")
        
    
    if MAD < VarianceMin:
        return False, MAD
    else:
        return True, MAD
    
def Overheat_Check(EventData, Dataset, Catalogue,Section):
    """
    Checks for overheat (Missing data).
    
    Parameters:
    - EventData: The list of triggers for the event, list of [M,3].
    - Dataset: The file location.
    - Catalogue: The catalogue location.
    - Section: The specific section, though this is actually unnecessary and can be called from attributes. TO DO LATER.
    
    Returns True/False if Not_Error/Error
    """
    with h5.File(f'{Dataset}','r') as PulledFile:
        Times = PulledFile['Data']['TimeData'][f'Section {Section}']
        Pulserate = PulledFile['Data']['RootInfo']['Acquisition'].attrs['PulseRate']

        Arbstart = int(EventData[40,1]-(Pulserate))
        Arbend = int(EventData[40,2]+(Pulserate))
        Timecheck = np.gradient(Times[Arbstart:Arbend],2000) #If the time values suddenly jump, their derivative is a significant quantity.
                                                             #This all just checks for that.
        for i in Timecheck:
            if i > 50:
                print("space found")
                return False
        
        #Pull the first and last trigger, check between them. If there's a "Gap" where the derivative of Times is huge, return False
        return True
    
def RangeCalc(eventdata,Pulserate):
    """
    Function stops automatic range generation from going out of bounds. 
    
    Parameters:
    - eventdata: list of triggers for the event, list of [M,3], 
    - Pulserate: Pulserate of the DAS.
    
    Returns:
    -eventstart: estimated start of the event (1 second early), datetime object
    -eventend: estimated end of the event (2 seconds later), datetime object
    
    Recent Change: added sintela_to_datetime for both returns so that the format is datetime rather than sintela.
    """
    
    if eventdata[0,1]-(1*Pulserate) < 0:  #if the first trigger - 1 seconds is below 0, set to 0.
        eventstart = 0
    else:
        eventstart = eventdata[0,1]-(1*Pulserate) #set at that time. 
    if eventdata[-1,2]+(2*Pulserate) > (60*Pulserate): #If the last trigger + 2 seconds is above 60, set to 60.
        eventend = 60*Pulserate
    else:
        eventend = eventdata[-1,2]+(2*Pulserate) #set at that time. 
    #eventstart = sintela_to_datetime(eventstart) #Note to self, cannot store datetime objects in h5 files.
    #eventend = sintela_to_datetime(eventend)
    return eventstart,eventend
        
    
    

    
def AddingPositionData(Dataset,Positionstxt):
    
    """
    Adds the positions of the cable.
    
    Parameters:
    - Dataset: the dataset the positions are to be connected to.
    - Positionstxt: txt file containing the positions of the cable in columns of X,Y,Z (In coordinates from an origin, I forget which TO DO.)
    
    Returns nothing
    """
    
    with h5.File(f'{Dataset}','a') as PulledFile:
        #if 'Cable' in PulledFile['Data']:
        #    Cableset = PulledFile['Data']['Cable']
        #else:
        Cableset = PulledFile['Data'].create_group('Cable')
        
        data = np.loadtxt(f"{Positionstxt}")

        col1 = data[:, 0]
        col2 = data[:, 1]
        col3 = data[:, 2]
        
        X = Cableset.create_dataset("CableX",data = col1)
        Y = Cableset.create_dataset("CableY",data = col2)
        Z = Cableset.create_dataset("CableZ",data = col3)
        
        X = X[:]
        Y = Y[:]
        Z = Z[:]
        global channel_locations_NY 
        channel_locations = np.array([X,Y,Z])

## Wave picker code

In [665]:
%%writefile P-picker.py

"""
Manually pick wave moveout for events.

Parameters:
TO DO


"""


import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import datetime
import pickle #dictionaries!
import scipy
from scipy import signal #For data cleanup
import obspy
from obspy.core import UTCDateTime #MAYBE useful.
from obspy.core import read
from obspy.signal import trigger #For triggering, required.


def sintela_to_datetime(sintela_times):
    '''
    Returns an array of datetime.datetime 
    Function provided by John-Morgand
    ''' 
    days1970 = datetime.date(1970, 1, 1).toordinal() #total days from 0

    # Vectorize everything
    converttime = np.vectorize(datetime.datetime.fromordinal)
    addday_lambda = lambda x : datetime.timedelta(days=x)
    adddays = np.vectorize(addday_lambda )
    
    day = days1970 + sintela_times/1e6/60/60/24 #Convert Sintela times to days from microseconds
    thisDateTime = converttime(np.floor(day).astype(int))
    dayFraction = day-np.floor(day)
    thisDateTime = thisDateTime + adddays(dayFraction)

    return thisDateTime


def getpwaves(Event, Catalogue, Dataset, Sensitivity = .05):
    
    """
    This function allows the user to pick p waves for a given event
    
    
    """
    PlotEvent(Event,Catalogue,Dataset,Sensitivity = Sensitivity,doseconds=False) #Plot the event, so that we can take picks.
    fig = plt.gcf() # gets last figure (so that the clicker knows what to look at)
    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    kid = fig.canvas.mpl_connect('key_press_event', onkey)
    global Cont #Janky function so user can take as many points as they want
    print("Shift left click to take a point, shift right click to remove it. Enter to end.")
    while Cont == True:
        plt.waitforbuttonpress()
        fig.canvas.draw_idle()
    fig.canvas.mpl_disconnect(cid)
    fig.canvas.mpl_disconnect(kid)# Disconnect the click handler
    plt.close()
    
    #Instead of return, maybe it should just save them directly to the event file? That might be the simplest, especially if I'm running this independent to the catalogue function.
    return PointsMaster
    
    
def onclick(event):
    if event.key != 'shift':  # Only handle clicks if Shift is held
        return
    print("Clicked!")
    if event.xdata is None or event.ydata is None:
        print("Click in the box, numbskull.")
        return
    if event.button == 1:
        print(f"Point recorded: ({event.xdata:.2f}, {event.ydata:.2f})")
        PointsMaster.append((event.xdata, event.ydata))

        Pointy = plt.scatter(event.xdata, event.ydata, color='red', s=10)
        PlottedPoints.append(Pointy)
        return
    elif event.button == 3 and PointsMaster != []:
        # Shift+Right-click to remove last point
        print(f"Removed last point: {PointsMaster[-1]}")
        point = PointsMaster.pop()
        Pointy = PlottedPoints.pop()
        Pointy.remove()
    else:
        return


#def onclick(event):
##    if collecting and event.key == 'shift' and event.inaxes:
 #       x, y = event.xdata, event.ydata
 #       PointsMaster.append((x, y))
 #       print(f"Point selected: ({x:.2f}, {y:.2f})")
def onkey(event):
    global Cont
    if event.key == 'enter':
        print("Enter pressed — stopping point collection.")
        Cont = False
        plt.close()  # Close the figure to exit the event loop



def Detect(Section,Dataset,Sensitivity=.05,Lowpass=1,Highpass=50,YLowerLim=None,YUpperLim=None, doseconds = True, show=True):
    """
    Generates a pcolormesh of the desired Section with a designated parameters
        
    Parameters:
    - Section: The section to plot
    - Dataset: The file location
    - Sensitivity: The sensitivity (in radians)
    - Lowpass: for Lowpass filter (Hz)
    - Highpass: for Highpass filter (Hz)
    - YLowerLim: Lower limit ()
    - YUpperLim: Upper limit ()
    - doseconds: If true, then the time axis is in seconds, if false it's datetime. 
    - show: Whether the plotted image should be shown.
    
    Returns nothing
        
    """
    #TO DO: Add an input to allow changing of ACTUAL FILTERING, not just highpass/lowpass.
    #TO DO: Make limits a tuple for simplification.
    #TO DO: Allow option to convert or not to convert to seconds, specifically for p wave picks... I need datetimes.
    
    Smin = -Sensitivity
    Smax = Sensitivity
    if YLowerLim == None:
        YLowerLim = 0
    if YUpperLim == None:
        YUpperLim = 60
    with h5.File(f'{Dataset}','r') as PulledFile: #WARNING: Generalize this, so that it can be from the "selected" catalogue
        DATA_TEMP = PulledFile['Data']['PosData'][f'Section {Section}']
        TIME_TEMP = PulledFile['Data']['TimeData'][f'Section {Section}']
        INFO = PulledFile['Data']['RootInfo']
        PR = INFO['Acquisition'].attrs['PulseRate']
    
        channels = np.arange(0,DATA_TEMP.shape[1],1)
        seconds = np.arange(0,DATA_TEMP.shape[0],1)
        Time = sintela_to_datetime(TIME_TEMP[:,]) 
        
    Seconds = [(T-Time[0]).total_seconds() for T in Time] #Super curious list comprehension function, I don't really get how it works but it's neat, "Vectorizes" the set of data and subtracts the time from each, which produces an array of seconds.
    #Seconds = [x for x in Seconds if YLowerLim <= x <= YUpperLim]

    sos = scipy.signal.butter(10, [Lowpass,Highpass], 'bp', fs = PR, output='sos') #WARNING WARNING: INFO['Acquisition'] IS NOT A GENERALIZED FUNCTION, LACK OF VERSATALITY!!!!
    filtered = signal.sosfiltfilt(sos,DATA_TEMP,axis=0)

    Seconds = np.round(np.array(Seconds),4)
    CorrectedLowerLim = np.where(Seconds == float(YLowerLim))
    CorrectedUpperLim = np.where(Seconds == float(YUpperLim))
    n = YLowerLim
    m = YUpperLim
    count = 0
    max_iters = 20
    while CorrectedLowerLim[0].size == 0:
        n = n - .01
        CorrectedLowerLim = np.where(Seconds == float(n))
        count += 1
        if count >= max_iters:
            n = YLowerLim
            count = 0
            while CorrectedLowerLim[0].size == 0:
                n = n + .01
                CorrectedLowerLim = np.where(Seconds == float(n))
                count +=1
                if count >= max_iters:
                    break
        if count >= max_iters:
            CorrectedLowerLim = (np.array([0]),)
            break
            
    count = 0
    while CorrectedUpperLim[0].size == 0:
        m = m + .01
        CorrectedUpperLim = np.where(Seconds == float(m))
        count += 1
        if count >= max_iters:
            m = YUpperLim
            count = 0
            while CorrectedUpperLim[0].size == 0:
                m = m - .01
                CorrectedUpperLim = np.where(Seconds == float(m))
                count +=1
                if count >= max_iters:
                    break
        if count >= max_iters:
            CorrectedUpperLim = (np.array([120000]),)
            break
    
    if doseconds == True:
        
    
        Seconds = Seconds[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0]]
        
        filtered = filtered[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0],:]
        
        fig,ax = plt.subplots()
        smesh = ax.pcolormesh(channels, Seconds, filtered, vmin = Smin, vmax = Smax, cmap = 'seismic',shading='nearest')
        plt.xlabel('channels')
        plt.ylabel('seconds')
        #ax.set_ylim(YLowerLim, YUpperLim)
        fig.colorbar(smesh,ax=ax)

    else:
        Time = Time[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0]]
        filtered = filtered[CorrectedLowerLim[0][0]:CorrectedUpperLim[0][0],:]
    
        fig,ax = plt.subplots()
        smesh = ax.pcolormesh(channels,Time,filtered, vmin = Smin, vmax = Smax, cmap = 'seismic',shading='nearest')
        plt.xlabel('channels')
        plt.ylabel('datetime')
        #ax.yaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S.%f')) 
        #ax.set_ylim(YLowerLim, YUpperLim)
        bar = fig.colorbar(smesh,ax=ax)
        bar.set_label("phase shift (radians)")
    if show==True:
        plt.show()
    return
    
    
def PlotEvent(Event,Catalogue,Dataset,Sensitivity=.05,Lowpass = None,Highpass = None,ShowTrig=False,ShowPick=False,Type=None, savefigs = False, folder=None,doseconds=True): #Plots an event from a catalogue.
    """
    Plots an event from the catalogue using its range.
    
    Parameters:
    - Event
    - Catalogue
    - Dataset
    - Sensitivity: The sensitivity (in radians)
    - Lowpass: for Lowpass filter (Hz)
    - Highpass: for Highpass filter (Hz)
    - ShowTrig: Show the triggers?
    - ShowPick: Show the automatic picks? (CURRENTLY DEPRECATED)
    - Type: Positives, Negatives, All?
    - savefigs: should figures be saved in the given path to folder?
    - folder: path to folder to save figures to
    - doseconds: If true, then the time axis is in seconds, if false it's datetime. 
    
    Returns nothing
    
    """
    with h5.File(Dataset,'r') as PulledFile:
        Pulserate = PulledFile['Data']['RootInfo']['Acquisition'].attrs['PulseRate']
        
        if Lowpass == None:
            Lowpass = PulledFile['Catalogues'][Catalogue].attrs['Lowpass']
        if Highpass == None:
            Highpass = PulledFile['Catalogues'][Catalogue].attrs['Highpass']        
    
    
        MyEvent = PulledFile['Catalogues'][Catalogue][f'Event {Event}']
        Section = PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Section']
        Range = PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Range']
        time = PulledFile['Data']['TimeData'][f'Section {Section}']
        Timeinit = sintela_to_datetime(time[0])
        doplot = None
       
        if Type == None:
            doplot = True
        elif Type == 'positive':
            if PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Error'] == False:
                doplot = True
            else:
                doplot = False
        elif Type == 'negative':
            if PulledFile['Catalogues'][Catalogue][f'Event {Event}'].attrs['Error'] == True:
                doplot = True
            else:
                doplot = False
        else:
            print("Nonvalid Type Parameter")
        if doplot == True:
            Detect(Section,Dataset,Sensitivity = Sensitivity,Lowpass = Lowpass,Highpass = Highpass,YLowerLim=(Range[0]/Pulserate),YUpperLim=(Range[1]/Pulserate),show=False,doseconds=doseconds)
            if ShowTrig==True:
                MyTrig = MyEvent['Triggers']
                MyTrig = MyTrig[:,:]
                plt.scatter(MyTrig[:,0],MyTrig[:,2]/Pulserate,s=7, alpha=.5)
                plt.scatter(MyTrig[:,0],MyTrig[:,1]/Pulserate,s=7, alpha=.5)
            if ShowPick==True:
                MyPick = MyEvent['P_waves']
                MyPick = MyPick[:,:]
                plt.scatter(MyPick[:,0],MyPick[:,1]/Pulserate,s=7, alpha=1)
            fig = plt.gcf()
            plt.title(f"Event {Event} in section {Section}, at {Timeinit}")
            #plt.show()
            if savefigs == True:
                os.makedirs(folder, exist_ok=True) 
                save_path = os.path.join(folder, f"Event_{Event}.png")
                plt.savefig(save_path)
    

PointsMaster = []
PlottedPoints = []
Cont = True

#First, set the initial data.
Dataset =  "/home/nyandell/Catalogue_Eastwind" #input("Enter File Path:")
Catalogue = "1-50 band sensitive search"#input("Enter Catalogue:")
Event = input("Enter Event Range:")
Type = input("positives/negatives/all?:")
#Event = list(map(int, Notyettuple.split(',')))
#for u in  Event:
#with h5.File(f'{Dataset}','r') as PulledFile:
    #CurrentGraph = PulledFile['Catalogues'][f'{Catalogue}']['Eventlist'][f'Event {Event}']
    #PlotEvent(Event,Catalogue,File)
if Event == 'all':
    with h5.File(Dataset,'r') as PulledFile:
        Totalnum = PulledFile['Catalogues'][Catalogue].attrs['Total']
        Event = (0,Totalnum)
else:
    temptup = Event
    temptup = temptup.strip("()")           # Remove parentheses
    parts = temptup.split(",")        # Split into elements
    Event = tuple(int(x) for x in parts)  # Convert to tuple of ints
for i in range(Event[0],Event[1]):
    tempcheck = False
    if Type != "all":
        if Type == "positives":
            Type = False
        elif Type == "negatives":
            Type = True
        with h5.File(Dataset,'r') as PulledFile:
            if PulledFile['Catalogues'][Catalogue][f'Event {i}'].attrs['Error']==Type:
                tempcheck = True
            else:
                None
        if tempcheck == True:
            print(f"pwaving {i}")
            getpwaves(i,Catalogue,Dataset,Sensitivity = .05) #I have this purposefully to avoid having the dataset already open, it's good to close it before calling the other function.
        else:
            print("skipping")
    else:
        getpwaves(i,Catalogue,Dataset,Sensitivity = .05)
    print(PointsMaster)
    #PointsMaster = [int(np.round(x)) for x in PointsMaster]
        
    with h5.File(Dataset,'a') as PulledFile:
        EventGroup = PulledFile['Catalogues'][Catalogue][f'Event {i}']
        if "P_waves" in EventGroup: #FIX
            del EventGroup['P_waves']
        EventGroup.create_dataset("P_waves", data=PointsMaster)
        PointsMaster = []
        PlottedPoints = []
        Cont = True

        

 
    #Do Ppicker now?
    #save events as an array, then with a 'a' file read append them to to h5 under each respective event.
    
#Pull the data & create the graph, then save a 1d array of p wave pick tuples, (channel & time).

#So we have PointsMaster, NOTE IT IS A series of tuples of FLOATs. We should save this as a dataset in the event group. Then, tomorrow talk to JM about getting the funciton set up. 








Overwriting P-picker.py


## Currently Unused

In [ ]:
"""
def Baer_Picker(Events, Position, Pulserate, tdownmax=20, tupevent=60, thr1=5, thr2=10, preset_len=250, p_dur=250, return_cf1=False):
    #Run a baer picker, refer to obspy? I remember that having a good one.
    
    #with h5.File(f'{Dataset}', 'r') as PulledFile:   
    #Pulserate = PulledFile['Data']['RootInfo']['Acquisition'].attrs['PulseRate']
    #Position = PulledFile['Data']['PosData'][f'Section {Section}']

    #tupevent=100
    #thr1=12
    #thr2=17
    #tdownmax=100
    
    
    All_p_picks = []
    Arbstart = int(Events[0][1]-(4*Pulserate))   #Note to self: This is arbitrary and may not be valid.
    Arbend = int(Events[0][2]+(4*Pulserate))
    if Arbstart < 0:
        Arbstart = 0
    if Arbend > 60*Pulserate:
        Arbend = int(60*Pulserate)

    for i in Events:
        Channel, Trigon, Trigoff = i   #(Trigon-5):(Trigoff+5)
        TempData = Position[Arbstart:int(Arbend+(3*Pulserate)),Channel] #Is 5 enough? Is it too much? Bleh.
        #print(Trigon)
        #print(TempData)
        p_pick, phase_info = trigger.pk_baer(TempData,Pulserate,tdownmax,tupevent,thr1,thr2,preset_len,p_dur,return_cf=return_cf1)
        p_pick=int(p_pick)
        #p_pick = p_pick/2000
        temp = [Channel, p_pick + Arbstart]
        All_p_picks.append(temp)
        #All_p_picks = np.append(All_p_picks, temp)
    All_p_picks=np.array(All_p_picks)

    for Channels in range(351):
        #Channel, Trigon, Trigoff = i   #(Trigon-5):(Trigoff+5)
        #print(Arbstart)
        #print(Arbend)
        TempData = Position[Arbstart:Arbend,Channels] #Is 5 enough? Is it too much? Bleh.
        #print(Trigon)
        #print(TempData)
        p_pick, phase_info = trigger.pk_baer(TempData,Pulserate,tdownmax,tupevent,thr1,thr2,preset_len,p_dur,return_cf=return_cf1)
        p_pick=int(p_pick)
        #p_pick = p_pick/2000
        temp = [Channels, p_pick + Arbstart]
        All_p_picks.append(temp)
        #All_p_picks = np.append(All_p_picks, temp)
    All_p_picks=np.array(All_p_picks)
        
        
    return All_p_picks #also return phase info maybe????????? Seems like it might be important.
        
        #extract the pos data
    
        #Using EventData, determine where to start the baer picker. 3 seconds before the trigger on maybe? Have it end 3 seconds after trigger off?
    
        #Record into a 2d array of (0,:) channels (1,:) baer pick.
        #This array will be returned.
     
    #p_pick, phase_info = trigger.pk_baer(DATA, Pulserate, 20, 60, 7.0, 12.0, 150, 150)
    #pk_baer(DATA, SampleRate, tdownmax, tupevent, thr1, thr2, preset_len, p_dur, return_cf=False)

    
    
    #return None
    
"""

In [11]:
def get_gaps(time_list, attrs, t_format=None):
    '''
    creates list with start and end times of data gaps
    Function provided by Dominik
    '''
    
    gap_list = []
    dt_eq = 1/attrs['PulseRate']*1e6
    for i in range(len(time_list)-1):
        gap_list.append((time_list[i][-1]+dt_eq, time_list[i+1][0]-dt_eq)) # times of gaps
    if t_format=='datetime':
        gap_list = [(sintela_to_datetime(s), sintela_to_datetime(e)) for (s,e) in gap_list]
    return gap_list

In [19]:
def PlotTrigs(Events, Section): #UNUSEDDDDDD
    with h5.File('/home/nyandell/CATALOGUE_EASTWIND/TEST_DELETE','r') as PulledFile:
        CurrentData = PulledFile['Data']['PosData'][f'Section {Section}']
        #data_norm = 
        #times_sliced = 
        
        fig, ax = plt.subplots(2,1,figsize=(6.4*2,2*4.8), gridspec_kw={'height_ratios': [0.2, 0.8]})
        print("made stuff")
        ax[0].plot(CurrentData, label='QJT03 vertical')
        ax[0].set_xlim(0, len(CurrentData))
        ax[0].set_xticks([])
        ax[0].set_yticks([])
        ax[0].legend()
        print("plotted")
        ax[1].imshow(CurrentData, aspect='auto',
             origin='lower',
             vmin=np.percentile(CurrentData[~np.isnan(CurrentData)],5),
             vmax=np.percentile(CurrentData[~np.isnan(CurrentData)],95),
             extent=[0, CurrentData.shape[0],
                  attrs['StartLocusIndex'], attrs['StartLocusIndex']+data_norm.shape[1]],
             cmap='RdBu_r',
             interpolation='none',
             )
        print("plotted2")
        cmap = plt.get_cmap('tab20')
        for i, ev in enumerate(events):
            ax[1].scatter(ev[:,2], ev[:,0], color=cmap((2*i+1)%20), alpha=0.2) # Trigger on
            ax[1].scatter(ev[:,1], ev[:,0], color=cmap((2*i)%20), alpha=0.2) # Trigger off

        ax[1].set_ylabel('DAS Channel Number') 
        ax[1].set_xlabel('Samples [{}Hz]'.format(attrs['PulseRate']))

        fig.subplots_adjust(hspace=0.02)
        print("done")


In [ ]:
def Tracing(Dataset,Section,Channel,Low,High,Start=None,End=None): #I'm unsure I use this function, ever. Does it even work anymore?
    
    """
    Section: The desired Section(s? maybe eventually?)
    
    Channel: The channel the trace is in.
    
    Low: Lowpass filter
    
    High: Highpass filter
    
    Start: Starting point
    
    End: Ending point
    
    Status: Completeish, basic functionality achieved. 
    
    """

    if Start == None:
        Start = 0
    if End == None:
        End = 60
        
    with h5.File(f'{Dataset}','r') as PulledFile:
        DATA_TEMP = PulledFile['Data']['PosData'][f'Section {Section}']
        #TIME_TEMP = PulledFile['Data']['TimeData'][f'Section {Section}']
        INFO = PulledFile['Data']['RootInfo']
        
        Pulserate = INFO['Acquisition'].attrs['PulseRate']
        stats = obspy.core.trace.Stats() 
        stats.sampling_rate = Pulserate 
        stats.npts = len(DATA_TEMP[:,Channel])
        stats.starttime = DATA_TEMP.attrs['PartStartTime']
        stats.network = 'N/A'
        stats.station = 'Eastwind'
        
    Sos = scipy.signal.butter(10, [Low,High], 'bp', fs = Pulserate, output='sos')
    Filtered = signal.sosfiltfilt(Sos,DATA_TEMP,axis=0)
    FilteredChannel = Filtered[:,Channel]

    t1 = stats.starttime + Start  # Start seconds after start
    t2 = stats.starttime + End  # End seconds after start
    
    #print(f'{t1} and {t2} are the start and end, from {Start} and {End}')  Possibly no longer needed testing line
    TestTrace = obspy.Trace(FilteredChannel,header=stats.copy())
    #TestTrace.plot(starttime=t1, endtime=t2,title=f"Filtered Channel {Channel}, Section {Section}",show=True)
    return TestTrace

# Workspace

In [531]:
DataInit("/home/nyandell/Catalogue_Eastwind", "/data/fast1/Eastwind_2425/Eastwind_decimator/Eastwind_decimator/*", "/home/nyandell/Eastwind_channel_locations.txt")


File does not exist, creating.
Corrupted file: /data/fast1/Eastwind_2425/Eastwind_decimator/Eastwind_decimator/Eastwind_decimator_2024-12-18_11.19.00_UTC.h5 - Error: Unable to synchronously open file (bad object header version number)


In [24]:
print(currentfile_NY)

/home/nyandell/Catalogue_Eastwind


In [200]:
#with h5.File(currentfile_NY,'r') as PulledFile:
#    print(PulledFile['Data']['PosData'].keys())

SetCatalogue('1-50 band sensitive search')
SetFile("/home/nyandell/Catalogue_Eastwind")

In [534]:

Catalogue_Classic('1-50 band sensitive search',(0,754),1,50,.5,3.0,TrigOn=2.3,TrigOff=1.5,tdownmax=160, tupevent=100, thr1=10, thr2=5, preset_len=4000, p_dur=2000,)



Just started Section 0
1 events so far, 0 positive and 1 negative
Just started Section 1
1 events so far, 0 positive and 1 negative
Just started Section 2
1 events so far, 0 positive and 1 negative
Just started Section 3
1 events so far, 0 positive and 1 negative
Just started Section 4
2 events so far, 1 positive and 1 negative
Just started Section 5
2 events so far, 1 positive and 1 negative
Just started Section 6
2 events so far, 1 positive and 1 negative
Just started Section 7
2 events so far, 1 positive and 1 negative
Just started Section 8
2 events so far, 1 positive and 1 negative
Just started Section 9
2 events so far, 1 positive and 1 negative
Just started Section 10
2 events so far, 1 positive and 1 negative
Just started Section 11
2 events so far, 1 positive and 1 negative
Just started Section 12
2 events so far, 1 positive and 1 negative
Just started Section 13
2 events so far, 1 positive and 1 negative
Just started Section 14
2 events so far, 1 positive and 1 negative
Just 

Just started Section 123
7 events so far, 6 positive and 1 negative
Just started Section 124
7 events so far, 6 positive and 1 negative
Just started Section 125
8 events so far, 7 positive and 1 negative
Just started Section 126
8 events so far, 7 positive and 1 negative
Just started Section 127
8 events so far, 7 positive and 1 negative
Just started Section 128
8 events so far, 7 positive and 1 negative
Just started Section 129
8 events so far, 7 positive and 1 negative
Just started Section 130
8 events so far, 7 positive and 1 negative
Just started Section 131
8 events so far, 7 positive and 1 negative
Just started Section 132
8 events so far, 7 positive and 1 negative
Just started Section 133
8 events so far, 7 positive and 1 negative
Just started Section 134
8 events so far, 7 positive and 1 negative
Just started Section 135
8 events so far, 7 positive and 1 negative
Just started Section 136
8 events so far, 7 positive and 1 negative
Just started Section 137
8 events so far, 7 posi

space found
space found
space found
space found
space found
space found
Just started Section 222
141 events so far, 10 positive and 131 negative
space found
space found
space found
space found
Just started Section 223
145 events so far, 10 positive and 135 negative
space found
space found
space found
space found
space found
Just started Section 224
150 events so far, 10 positive and 140 negative
space found
Just started Section 225
151 events so far, 10 positive and 141 negative
space found
space found
space found
space found
space found
Just started Section 226
156 events so far, 10 positive and 146 negative
space found
space found
Just started Section 227
158 events so far, 10 positive and 148 negative
space found
Just started Section 228
159 events so far, 10 positive and 149 negative
space found
space found
Just started Section 229
161 events so far, 10 positive and 151 negative
space found
space found
space found
space found
space found
space found
space found
Just started Section

Just started Section 320
228 events so far, 16 positive and 212 negative
Just started Section 321
228 events so far, 16 positive and 212 negative
Just started Section 322
229 events so far, 17 positive and 212 negative
Just started Section 323
229 events so far, 17 positive and 212 negative
Just started Section 324
229 events so far, 17 positive and 212 negative
Just started Section 325
229 events so far, 17 positive and 212 negative
Just started Section 326
229 events so far, 17 positive and 212 negative
Just started Section 327
229 events so far, 17 positive and 212 negative
Just started Section 328
229 events so far, 17 positive and 212 negative
Just started Section 329
230 events so far, 18 positive and 212 negative
Just started Section 330
231 events so far, 19 positive and 212 negative
Just started Section 331
231 events so far, 19 positive and 212 negative
Just started Section 332
232 events so far, 20 positive and 212 negative
Just started Section 333
232 events so far, 20 posi

Just started Section 433
279 events so far, 63 positive and 216 negative
Just started Section 434
281 events so far, 65 positive and 216 negative
Just started Section 435
282 events so far, 66 positive and 216 negative
Just started Section 436
282 events so far, 66 positive and 216 negative
Just started Section 437
282 events so far, 66 positive and 216 negative
Just started Section 438
283 events so far, 67 positive and 216 negative
Just started Section 439
285 events so far, 69 positive and 216 negative
Just started Section 440
286 events so far, 70 positive and 216 negative
Just started Section 441
286 events so far, 70 positive and 216 negative
Just started Section 442
286 events so far, 70 positive and 216 negative
Just started Section 443
287 events so far, 71 positive and 216 negative
Just started Section 444
288 events so far, 72 positive and 216 negative
Just started Section 445
288 events so far, 72 positive and 216 negative
Just started Section 446
288 events so far, 72 posi

Just started Section 546
307 events so far, 91 positive and 216 negative
Just started Section 547
307 events so far, 91 positive and 216 negative
Just started Section 548
307 events so far, 91 positive and 216 negative
Just started Section 549
307 events so far, 91 positive and 216 negative
Just started Section 550
309 events so far, 93 positive and 216 negative
Just started Section 551
309 events so far, 93 positive and 216 negative
Just started Section 552
309 events so far, 93 positive and 216 negative
Just started Section 553
309 events so far, 93 positive and 216 negative
Just started Section 554
309 events so far, 93 positive and 216 negative
Just started Section 555
309 events so far, 93 positive and 216 negative
Just started Section 556
309 events so far, 93 positive and 216 negative
Just started Section 557
310 events so far, 94 positive and 216 negative
Just started Section 558
310 events so far, 94 positive and 216 negative
Just started Section 559
310 events so far, 94 posi

Just started Section 658
322 events so far, 106 positive and 216 negative
Just started Section 659
322 events so far, 106 positive and 216 negative
Just started Section 660
322 events so far, 106 positive and 216 negative
Just started Section 661
322 events so far, 106 positive and 216 negative
Just started Section 662
322 events so far, 106 positive and 216 negative
Just started Section 663
322 events so far, 106 positive and 216 negative
Just started Section 664
322 events so far, 106 positive and 216 negative
Just started Section 665
322 events so far, 106 positive and 216 negative
Just started Section 666
322 events so far, 106 positive and 216 negative
Just started Section 667
322 events so far, 106 positive and 216 negative
Just started Section 668
322 events so far, 106 positive and 216 negative
Just started Section 669
323 events so far, 107 positive and 216 negative
Just started Section 670
323 events so far, 107 positive and 216 negative
Just started Section 671
323 events so

In [209]:
with h5.File(f'{currentfile_NY}','r') as PulledFile: #WARNING: Generalize this, so that it can be from the "selected" catalogue
    DATA_TEMP = PulledFile['Data']['PosData'][f'Section 1']
    TIME_TEMP = PulledFile['Data']['TimeData'][f'Section 1']
    INFO = PulledFile['Data']['RootInfo']
    PR = INFO['Acquisition'].attrs['PulseRate']
    Frsh = PulledFile['Catalogues']['1-50 band sensitive search']['Event 0'].attrs['Range']

    channels = np.arange(0,DATA_TEMP.shape[1],1)
    seconds = np.arange(0,DATA_TEMP.shape[0],1)
    Time = sintela_to_datetime(TIME_TEMP[:,]) 
    
    #Time2 = mdates.date2num(Time)
    #print(Time2)
    #Seconds = [(T-Time[0]).total_seconds() for T in Time] 
    
    #print([Time,seconds])
    #plt.plot(Time2, Time)
    #plot()
    print(Frsh[0])
    
    

44811.0


In [336]:
RANGE = 0
with h5.File(f'{currentfile_NY}','r') as PulledFile:
    i=0
    for i in range(RANGE):
        Frsh = PulledFile['Catalogues']['1-50 band sensitive search'][f'Event {i}']
        PlotEvent(i,'1-50 band sensitive search',currentfile_NY)
    

In [238]:
#

"""def move_event(event,catalogue, error=False): #Do NOT use this yet, this requires each event ot have unique ID!!!!!!
    with h5.File(f'{currentfile_NY}','a') as PulledFile:
        if error==False:
            new = PulledFile['Catalogues'][f'{catalogue}']['ErrorList'].creategroup(f'Error {event}')
            print(new)
            print(new.attrs)
            tempatt = PulledFile['Catalogues'][f'{catalogue}']['EventList'][f'Event {event}'].attrs
            new.attrs = tempatt
            print(new.attrs)
            tempdat = PulledFile['Catalogues'][f'{catalogue}']['EventList'][f'Event {event}']
            new = tempdat
            print(new)
            #move the event from the Eventlist to the Errorlist    
        else:
            None
        #move the event from the Errorlist to the Eventlist"""


#UNUSED, NO LONGER NECESSARY DUE TO FORMAT CHANGE

"def move_event(event,catalogue, error=False): #Do NOT use this yet, this requires each event ot have unique ID!!!!!!\n    with h5.File(f'{currentfile_NY}','a') as PulledFile:\n        if error==False:\n            new = PulledFile['Catalogues'][f'{catalogue}']['ErrorList'].creategroup(f'Error {event}')\n            print(new)\n            print(new.attrs)\n            tempatt = PulledFile['Catalogues'][f'{catalogue}']['EventList'][f'Event {event}'].attrs\n            new.attrs = tempatt\n            print(new.attrs)\n            tempdat = PulledFile['Catalogues'][f'{catalogue}']['EventList'][f'Event {event}']\n            new = tempdat\n            print(new)\n            #move the event from the Eventlist to the Errorlist    \n        else:\n            None\n        #move the event from the Errorlist to the Eventlist"

In [547]:
with h5.File(f'{currentfile_NY}','r') as PulledFile:
    Wee = PulledFile['Catalogues'][currentcatalogue_NY]["Event 1"].attrs['Range']
    print(Wee)

[70249. 80116.]


In [671]:
#Detect(339,YLowerLim=33.5,YUpperLim=35)

In [670]:
#for i in range(754):
    #PlotEvent(i,Type='positive')

In [669]:
#for i in range(754):
    #PlotEvent(i,Type='negative')

In [588]:
with h5.File(f'{currentfile_NY}','r') as PulledFile:
    print(PulledFile.keys())

<KeysViewHDF5 ['Catalogues', 'Data']>


In [599]:
Testwaves,Testdata = get_subsection(1)


print(Testwaves)
print(Testdata)

[[   28.15065179    42.47597768    43.31864391    56.80130357
     71.9692957     86.29462159    98.93461503   112.41727469
    113.25994092   124.2146019    123.37193567   131.79859796
    141.91059271   141.91059271   160.44924974   178.14524055
    197.52656382   198.36923005   215.22255463   214.3798884
    232.07587921   245.55853888]
 [20075.20737039 20075.20736978 20075.20736978 20075.20736917
  20075.20736867 20075.20736811 20075.20736756 20075.20736711
  20075.20736711 20075.20736667 20075.20736667 20075.20736656
  20075.20736672 20075.20736667 20075.20736684 20075.20736678
  20075.20736728 20075.20736728 20075.20736778 20075.20736778
  20075.20736839 20075.20736895]]
[[ 0.01389276 -0.49923035 -0.18184893 ... -0.00692169  0.00483025
  -0.00989548]
 [ 0.00508235 -0.50132695 -0.15404988 ... -0.00712244  0.00533848
  -0.01007052]
 [-0.00361549 -0.50103434 -0.12056604 ... -0.00730737  0.00587594
  -0.0103236 ]
 ...
 [-0.0256655   0.26296636 -0.65817919 ...  0.00449214 -0.00556353


In [613]:
describeevents((0,10))

Event 0 has the attributes ['Error', 'Range', 'Section'] with the values [np.True_, array([46811., 57347.]), np.int64(0)]
Event 1 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([70249., 80116.]), np.int64(4)]
Event 2 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([50091., 61777.]), np.int64(35)]
Event 3 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([88933., 98671.]), np.int64(47)]
Event 4 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([66873., 77065.]), np.int64(81)]
Event 5 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([ 5815., 14808.]), np.int64(102)]
Event 6 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([61038., 71111.]), np.int64(113)]
Event 7 has the attributes ['Error', 'Range', 'Section'] with the values [np.False_, array([46890., 57128.]), np.int64(125)]
Event 8 

In [667]:
#for i in range(3):
    PlotEvent(i,folder="/home/nyandell/folderbox",savefigs=True)

IndentationError: unexpected indent (1170426786.py, line 2)

In [666]:
#PlotEvent(8)

In [655]:
autochange()

In [661]:
with h5.File("/home/nyandell/Catalogue_Eastwind","r") as PulledFile:
    pick = PulledFile['Catalogues']['1-50 band sensitive search'].attrs['Positives']
    pick2 = PulledFile['Catalogues']['1-50 band sensitive search'].attrs['Negatives']
    pickcheck = PulledFile['Catalogues']['1-50 band sensitive search'].attrs['Total']
    print(pick)
    print(pick2)
    print(pick+pick2)
    print(pickcheck)

88
258
346
346
